## Naive Bayes

<img src="http://www.cs.cornell.edu/courses/cs4780/2015fa/web/projects/03NaiveBayes/nb.png" alt="naive_bayes" style="width:350px;"/>

Naive Bayes methods are a set of supervised learning algorithms based on **applying Bayes’ theorem with the “naive” assumption of conditional independence between every pair of features** given the value of the class variable. Bayes’ theorem states the following relationship, given class variable $y$ and dependent feature vector $x_1$ through $x_n$, :

- $x_1, ..., x_n$ : feature
- $y$ : output class

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots x_n \mid y)}
                                 {P(x_1, \dots, x_n)}$$
                                 

Using the naive **conditional independence assumption** that

$$P(x_i | y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i | y),$$

for all $i$, this relationship is simplified to

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)}
                                 {P(x_1, \dots, x_n)}$$

Since $P(x_1,...,x_n)$ is constant given the input, we can use the following classification rule:

\begin{align}\begin{aligned}P(y \mid x_1, \dots, x_n) \propto P(y) \prod_{i=1}^{n} P(x_i \mid y)\\\Downarrow\\\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y),\end{aligned}\end{align}

and we can use Maximum A Posteriori (MAP) estimation to estimate $P(y)$ and $P(x_i | y)$; the former is then the relative frequency of class  in the training set.

## Compute Prior ($P(y)$)

Just count how many times each class came out, then normalize by sum of all counts

```python
class_prior = np.log(float(len(class_indices)) / len(train_Y))
```

## Design Choices for Naive Bayes ($P(x_i|y)$)

Only thing we need to decide when designing naive bayes classifier is how to represent $P(x_i|y)$.

There are several options depends on the type of each feature.

### Discrete-valued feature case ([Bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution))

Parameters to learn
- $P(i \mid y)$ : 해당 class가 0 과 1이 각각 몇번 나왔는지 세어준 후 전체 횟수로 나눠주면 됨


$$
P(x_i \mid y) = P(i \mid y) x_i + (1 - P(i \mid y)) (1 - x_i)
$$

<img src="http://bluehawk.monmouth.edu/rclayton/web-pages/s05-525/sapdg2-note.png" alt="bernoulli_distribution" style="width:250px;"/>

```python
from scipy.stats import bernoulli

class_means = np.mean(class_features, axis=0)
prob_function = bernoulli(class_means[idx]).pmf
```


### Continuous-valued feature case ([Gaussian distribution](https://en.wikipedia.org/wiki/Normal_distribution))

Parameters to learn
- $\mu$ : 해당 class의 평균을 구해주면 됨
- $\sigma$ : 해당 class의 분산을 구해주면 됨

$$
P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)
$$

<img src="http://images.books24x7.com/bookimages/id_5642/fig11-10.jpg" alt="gaussian_distribution" style="width:250px;"/>

```python
from scipy.stats import norm

class_means = np.mean(class_features, axis=0)
class_stds = np.std(class_features, axis=0)
prob_function = norm(class_means[idx], class_stds[idx]).pdf
```

Now, let's check the code

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
from operator import itemgetter

from scipy.stats import norm, bernoulli
import numpy as np

In [ ]:
def read_dataset(train_fname, test_fname, features_to_use=None):
    def _read_dataset(fname):
        X = []
        Y = []
        with open(fname, 'r') as fp:
            reader = csv.reader(fp)
            for idx, line in enumerate(reader):
                if idx == 0:
                    continue

                if features_to_use is None:
                    X.append(line[1:])
                else:
                    X.append(itemgetter(*features_to_use)(line[1:]))
                Y.append(line[0])
        X = np.array(X, np.float32)
        Y = np.array(Y, np.float32)
        return (X, Y)

    train_data = _read_dataset(train_fname)
    test_data = _read_dataset(test_fname)

    return train_data, test_data


def compute_accuracy(predictions, answers):
    assert len(predictions) == len(answers), \
        "Num predictions %d and num answers %d does not match" % (len(predictions), len(answers))

    num_correct = 0
    for prediction, answer in zip(predictions, answers):
        if prediction == answer:
            num_correct += 1
    return float(num_correct) / len(predictions)

In [ ]:
class NaiveBayesClassifier(object):
    def __init__(self, num_classes=2):
        self._num_classes = num_classes
        self._feature_prob_functions = {}
        self._class_prior = []

    def fit(self, train_X, train_Y, is_continuous):
        for class_id in range(self._num_classes):
            class_indices = np.where(train_Y == float(class_id))[0]
            class_features = np.take(train_X, class_indices, axis=0)
            class_means = np.mean(class_features, axis=0)
            class_stds = np.std(class_features, axis=0)
            class_prior = np.log(float(len(class_indices)) / len(train_Y))

            feature_prob_functions = []
            for idx, continuous in enumerate(is_continuous):
                if continuous:
                    prob_function = norm(class_means[idx], class_stds[idx]).pdf
                else:
                    prob_function = bernoulli(class_means[idx]).pmf
                feature_prob_functions.append(prob_function)
            self._feature_prob_functions[class_id] = feature_prob_functions
            self._class_prior.append(class_prior)

    def predict(self, test_X):
        predictions = []
        for test_x in test_X:
            output_prob = []
            for class_id in range(self._num_classes):
                cum_prob = self._class_prior[class_id]
                for x, prob_function in zip(test_x, self._feature_prob_functions[class_id]):
                    cum_prob += np.log(prob_function(x))
                output_prob.append(cum_prob)
            prediction = np.argmax(output_prob)
            predictions.append(prediction)
        return predictions
    
def run_naive_bayes(train_data, test_data, is_continuous):
    train_X, train_Y = train_data
    test_X, test_Y = test_data

    nb_classifier = NaiveBayesClassifier()
    nb_classifier.fit(train_X, train_Y, is_continuous)
    pred_Y = nb_classifier.predict(test_X)
    accuracy = compute_accuracy(pred_Y, test_Y)

    return accuracy

In [ ]:
train_data, test_data = read_dataset("./train_data.csv", "./test_data.csv")
is_continuous = [True] * 2 + [False] * 8 + [True] * 2
accuracy = run_naive_bayes(train_data, test_data, is_continuous)
print("Accuracy : {}".format(accuracy))

### References
- [Naive Bayes in scikit-learn](http://scikit-learn.org/stable/modules/naive_bayes.html)